# Batch run

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
import numpy as np

In [ ]:
trainX = np.load("../data/trainX.npy")
output_train = np.load("../data/output_train.npy")
validateX = np.load("../data/validateX.npy")
output_validate = np.load("../data/output_validate.npy")
testX = np.load("../data/testX.npy")
output_test = np.load("../data/output_test.npy")

In [ ]:
import os
import json
import csv

EPOCHS=10



# Define hyperparameter values
batch_sizes = [16, 32, 64]  # Batch sizes
input_shapes = [(10, trainX.shape[2]), (20, trainX.shape[2]), (30, trainX.shape[2])]  # Time steps
lstm_units_list = [16, 32]  # LSTM units

# CSV file to log results
csv_filename = "training_results.csv"
csv_header = ["Batch Size", "Time Steps", "LSTM Units", "Final Training Loss", "Final Validation Loss", "Epochs Trained"]

# If CSV file does not exist, create it and add a header
if not os.path.exists(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_header)

# Training Loop
run_id = 0
for batch_size in batch_sizes:
    for input_shape in input_shapes:
        for lstm_units in lstm_units_list:
            run_id += 1
            print(f"\nTraining Run {run_id}: batch_size={batch_size}, time_steps={input_shape[0]}, lstm_units={lstm_units}\n")

            with tf.device('CPU:0'):
                # Define Model
                model = Sequential()
                model.add(LSTM(lstm_units, input_shape=input_shape, return_sequences=True, activation='sigmoid'))
                model.add(TimeDistributed(Dense(1)))  # Output at each time step

                # Compile Model
                model.compile(optimizer=RMSprop(learning_rate=0.01), loss='mean_squared_error', metrics=['mse'])

                # Early Stopping (Stops if validation loss doesn't improve for 20 epochs)
                early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

            # Create directory for this run
            model_name = f"model_bs{batch_size}_ts{input_shape[0]}_units{lstm_units}"
            run_dir = f"./models/{model_name}/"
            os.makedirs(run_dir, exist_ok=True)



            # Train Model
            history = model.fit(trainX, output_train, epochs=EPOCHS, batch_size=batch_size, 
                                validation_data=(validateX, output_validate), 
                                verbose=2, callbacks=[early_stopping])

            # Save final model
            model.save(os.path.join(run_dir, f"{model_name}.keras"))

            # Save training history
            history_path = os.path.join(run_dir, f"{model_name}_history.json")
            with open(history_path, 'w') as f:
                json.dump(history.history, f)

            # Log results to CSV
            final_training_loss = history.history['loss'][-1]
            final_validation_loss = history.history['val_loss'][-1]
            epochs_trained = len(history.history['loss'])

            with open(csv_filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_size, input_shape[0], lstm_units, final_training_loss, final_validation_loss, epochs_trained])

            print(f"Model and history saved in {run_dir}")
            print(f"Results logged to {csv_filename}")


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import pandas as pd

# Load results CSV into a DataFrame
df = pd.read_csv("training_results.csv")

# Display the results sorted by lowest validation loss
df_sorted = df.sort_values(by="Final Validation Loss", ascending=True)
print(df_sorted)


In [ ]:
import json
import matplotlib.pyplot as plt

# Load training history
history_path = "./models/model_bs32_ts20_units64/model_bs32_ts20_units64_history.json"
with open(history_path, 'r') as f:
    history = json.load(f)

# Plot loss curves
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Performance')
plt.show()
